In [1]:
import pandas as pd
import numpy as np
import warnings
import pickle # Added for saving/loading
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from lightgbm import LGBMRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

warnings.filterwarnings('ignore')

# --- Log-transform function (Defined globally) ---
def log_transform(df, return_logged=False):
    df_log = df.copy()
    logged_columns = []
    for col in df_log.columns:
        # Check if > 70% of data is positive AND all data is positive
        if (df[col] > 0).sum() / len(df) > 0.7 and (df[col] > 0).all(): 
            df_log[col] = np.log1p(df[col])
            logged_columns.append(col)
    if return_logged: return df_log, logged_columns
    return df_log

# --- 1. Data Loading and Synthetic Fallback ---

try:
    data = pd.read_csv('hwc.csv')
    print(f"CSV loaded: Shape {data.shape}")
    if data.empty:
        raise ValueError("CSV empty—check file.")
except FileNotFoundError:
    print("File 'hwc.csv' not found—using synthetic fallback.")
    data = None
except Exception as e:
    print(f"Load error ({e})—using synthetic fallback.")
    data = None

if data is not None and not data.empty:
    unnecessary = [
        'P_NAME', 'S_NAME', 'S_NAME_HD', 'S_NAME_HIP', 'P_YEAR', 'P_UPDATE', 'P_DETECTION', 'P_DISCOVERY_FACILITY',
        'P_MASS_ORIGIN', 'S_MASS_ORIGIN', 'S_METALLICITY_ORIGIN', 'S_FE_H_ORIGIN',
        'S_VELOCITY_REFERENCE', 'S_VELOCITY_METHOD', 'S_VELOCITY_FLAG', 'S_VELOCITY_QUALITY', 'S_VELOCITY_SOURCE',
        'S_VELOCITY_RADIAL_ORIGIN', 'S_VELOCITY_RADIAL_REFERENCE', 'S_VELOCITY_RADIAL_METHOD', 'S_VELOCITY_RADIAL_FLAG',
        'S_VELOCITY_RADIAL_QUALITY', 'S_VELOCITY_RADIAL_SOURCE', 'S_VELOCITY_PROPER_MOTION_ORIGIN',
        'S_VELOCITY_PROPER_MOTION_REFERENCE', 'S_VELOCITY_PROPER_MOTION_METHOD', 'S_VELOCITY_PROPER_MOTION_FLAG',
        'S_VELOCITY_PROPER_MOTION_QUALITY', 'S_VELOCITY_PROPER_MOTION_SOURCE', 'S_VELOCITY_TANGENTIAL_ORIGIN',
        'S_VELOCITY_TANGENTIAL_REFERENCE', 'S_VELOCITY_TANGENTIAL_METHOD', 'S_VELOCITY_TANGENTIAL_FLAG',
        'S_VELOCITY_TANGENTIAL_QUALITY', 'S_VELOCITY_TANGENTIAL_SOURCE', 'S_VELOCITY_TOTAL_ORIGIN',
        'S_VELOCITY_TOTAL_REFERENCE', 'S_VELOCITY_TOTAL_METHOD', 'S_VELOCITY_TOTAL_FLAG', 'S_VELOCITY_TOTAL_QUALITY',
        'S_VELOCITY_TOTAL_SOURCE', 'S_TYPE', 'S_STELLAR_SPECIES', 'S_TYPE_SPECTRAL_TYPE', 'S_RA_STR', 'S_DEC_STR'
    ]
    data = data.drop(columns=[col for col in unnecessary if col in data.columns])
    data = data.apply(pd.to_numeric, errors='coerce')
    numeric_columns = data.select_dtypes(include=[np.number]).columns.tolist()
    data = data[numeric_columns]
else:
    print("Generating synthetic data (20 rows).")
    np.random.seed(42)
    n_samples = 20
    data = pd.DataFrame({
        'P_PERIOD': np.random.uniform(50, 1000, n_samples), 'P_RADIUS': np.random.uniform(0.5, 5.0, n_samples), 
        'P_SEMI_MAJOR_AXIS': np.random.uniform(0.1, 10.0, n_samples), 'P_ECCENTRICITY': np.random.uniform(0, 0.5, n_samples),
        'P_TEMP_EQUIL': np.random.uniform(100, 500, n_samples), 'S_TEMPERATURE': np.random.uniform(4000, 7000, n_samples),
        'S_MAG': np.random.uniform(5, 15, n_samples), 'P_DETECTION': np.random.choice([0, 1], n_samples),
        'P_INCLINATION': np.random.uniform(80, 100, n_samples), 'S_AGE': np.random.uniform(1, 10, n_samples),
        'P_VELOCITY': np.random.uniform(0, 50, n_samples), 'P_MASS': np.random.uniform(0.1, 10.0, n_samples),
        'P_DENSITY': np.random.uniform(0.5, 8.0, n_samples), 'S_MASS': np.random.uniform(0.5, 2.0, n_samples),
        'P_MASS_ERROR_MIN': np.random.uniform(0.01, 0.1, n_samples), 
        'S_LOG_G': np.random.uniform(4.0, 5.0, n_samples), 'S_LOG_LUM': np.random.uniform(-0.5, 1.0, n_samples)
    })
    for i in range(n_samples):
        # A simple synthetic relationship for mass
        data.loc[i, 'P_MASS'] = 1.2 * (data.loc[i, 'P_PERIOD'] / 365)**(2/3) * data.loc[i, 'P_SEMI_MAJOR_AXIS'] * (1 - data.loc[i, 'P_ECCENTRICITY']) + np.random.normal(0, 0.3)
    numeric_columns = data.columns.tolist()

# --- 2. Data Cleaning and Imputation ---

if data.empty or data.shape[1] == 0:
    raise ValueError("No valid data remaining after load/generation.")

missing_pct = (data.isnull().sum() / len(data)) * 100
high_missing_cols = missing_pct[missing_pct > 40].index.tolist()
data = data.drop(columns=high_missing_cols)
numeric_columns = [col for col in numeric_columns if col not in high_missing_cols]
data = data.fillna(data.mean(numeric_only=True))

# --- 3. Feature/Target Selection (WITH COMPOSITION HINT) ---

input_candidates = ['P_PERIOD', 'P_RADIUS', 'P_TEMP_EQUIL', 'S_TEMPERATURE', 'S_MASS']
input_columns = [col for col in input_candidates if col in numeric_columns]

all_cols_for_target_selection = [col for col in numeric_columns if col not in input_columns]
unstable_targets = [col for col in all_cols_for_target_selection if 'LIMIT' in col or 'ERROR' in col]
unstable_targets.extend(['S_RA', 'S_DEC', 'S_METALLICITY', 'S_ABIO_ZONE', 'P_DETECTION', 'S_MAG', 'S_AGE']) 
unstable_targets = list(set(unstable_targets)) 

target_columns = [col for col in all_cols_for_target_selection if col not in unstable_targets]

if len(input_columns) < 3 or len(target_columns) == 0:
    raise ValueError("Not enough inputs or stable targets to train the model.")

print(f"\n--- Model Configuration ---")
print(f"Inputs (Minimal Initial + Composition Hint, {len(input_columns) + 1}): {input_columns} + P_COMPOSITION_ROCKY")
print(f"Targets (Derived/Hard, {len(target_columns)}): P_ECCENTRICITY is included. Other examples: {target_columns[:5]}...")

# --- 4. Preprocessing and Feature Engineering ---

data_clipped = data.copy()
for col in numeric_columns:
    if data[col].notna().sum() > len(data) * 0.5:
        q99 = data[col].quantile(0.99)
        data_clipped[col] = np.clip(data_clipped[col], None, q99)

X_base = data_clipped[input_columns].apply(pd.to_numeric, errors='coerce').fillna(data_clipped[input_columns].mean())
y_base = data_clipped[target_columns].apply(pd.to_numeric, errors='coerce').fillna(data_clipped[target_columns].mean())

# Add P_COMPOSITION_ROCKY
if 'P_DENSITY' in data_clipped.columns:
    X_base['P_COMPOSITION_ROCKY'] = (data_clipped['P_DENSITY'] >= 3.0).astype(int)
    input_columns.append('P_COMPOSITION_ROCKY')
    print("Added P_COMPOSITION_ROCKY to training data based on P_DENSITY.")
else:
    # If P_DENSITY is missing, we must remove 'P_COMPOSITION_ROCKY' from the list if it was assumed.
    # For synthetic data, P_DENSITY exists, but this is a safety measure.
    print("Warning: P_COMPOSITION_ROCKY not added due to missing P_DENSITY in dataset.")

# Feature Engineering: Proxies
if 'P_TEMP_EQUIL' in data_clipped.columns:
    water_proxy = (data_clipped['P_TEMP_EQUIL'] >= 200) & (data_clipped['P_TEMP_EQUIL'] <= 400)
    X_base['water_potential_proxy'] = water_proxy.astype(int)
    if 'water_potential_proxy' not in input_columns: input_columns.append('water_potential_proxy')
    
# Log-transform
X = log_transform(X_base[input_columns], return_logged=False)
y, logged_columns = log_transform(y_base, return_logged=True)

X = X.dropna(thresh=X.shape[1] * 0.7)
y = y.loc[X.index]

# Train/Test Split and Scaling
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_scaled = scaler_X.fit_transform(X_train)
y_train_scaled = scaler_y.fit_transform(y_train)
X_test_scaled = scaler_X.transform(X_test)
y_test_scaled = scaler_y.transform(y_test)

# --- 5. Model Training and Saving to ONE .pkl File ---

print("\nTraining LightGBM MultiOutputRegressor (Fast Mode)...")
best_lgb = LGBMRegressor(random_state=42, verbose=-1, n_jobs=-1, n_estimators=150, learning_rate=0.08)
model = MultiOutputRegressor(best_lgb) 
model.fit(X_train_scaled, y_train_scaled)

# Store all necessary components in a single dictionary
ml_system = {
    'model': model,
    'scaler_X': scaler_X,
    'scaler_y': scaler_y,
    'X_train_columns': X_train.columns.tolist(), 
    'target_columns': target_columns, 
    'logged_columns': logged_columns
}

# Save the entire dictionary to a single file
try:
    with open('exoplanet_predictor_system.pkl', 'wb') as file:
        pickle.dump(ml_system, file)
    print("✅ System components saved to ONE file: 'exoplanet_predictor_system.pkl'")
except Exception as e:
    print(f"Error saving file: {e}")

# --- 6. Prediction Section (Fully Compatible with .pkl File) ---

def load_system(filename='exoplanet_predictor_system.pkl'):
    """Loads all model components from the saved .pkl file."""
    try:
        with open(filename, 'rb') as file:
            system = pickle.load(file)
        print(f"\n✅ Loaded model system from '{filename}'")
        print(f"Expected {len(system['X_train_columns'])} input features:")
        print(system['X_train_columns'])
        return system
    except Exception as e:
        print(f"❌ Failed to load system from {filename}: {e}")
        return None


def predict_exoplanet_properties(new_input_dict, system_dict):
    """Predicts exoplanet properties with auto feature alignment."""
    trained_model = system_dict['model']
    scaler_x = system_dict['scaler_X']
    scaler_y = system_dict['scaler_y']
    X_train_original_cols = system_dict['X_train_columns']
    logged_cols = system_dict['logged_columns']
    target_cols = system_dict['target_columns']

    new_data_raw = pd.DataFrame({k: [v] for k, v in new_input_dict.items()})
    X_new = pd.DataFrame(index=[0], columns=X_train_original_cols)

    # --- Auto-fill all required features ---
    missing_cols = []
    for col in X_train_original_cols:
        if col in new_data_raw.columns:
            X_new[col] = new_data_raw[col]
        elif col == 'water_potential_proxy' and 'P_TEMP_EQUIL' in new_data_raw.columns:
            temp_val = new_data_raw['P_TEMP_EQUIL'].iloc[0]
            X_new[col] = int(200 <= temp_val <= 400)
        elif col == 'P_COMPOSITION_ROCKY' and 'P_DENSITY' in new_data_raw.columns:
            X_new[col] = int(new_data_raw['P_DENSITY'].iloc[0] >= 3.0)
        else:
            X_new[col] = 0
            missing_cols.append(col)

    if missing_cols:
        print(f"⚠️ Missing columns (auto-filled with 0): {missing_cols}")

    X_new = X_new.apply(pd.to_numeric, errors='coerce').fillna(0)
    X_new_logged = log_transform(X_new)
    X_new_scaled = scaler_x.transform(X_new_logged)

    # --- Prediction ---
    y_pred_scaled = trained_model.predict(X_new_scaled)
    y_pred_orig = scaler_y.inverse_transform(y_pred_scaled)

    y_new_pred_df = pd.DataFrame(y_pred_orig, columns=target_cols)

    # --- Reverse log-transform for logged targets ---
    for col in logged_cols:
        if col in y_new_pred_df.columns:
            y_new_pred_df[col] = np.expm1(np.clip(y_new_pred_df[col], a_min=0, a_max=None))

    return X_new, y_new_pred_df


# --- 7. Accuracy Evaluation ---
y_pred_scaled = model.predict(X_test_scaled)
y_test_orig = scaler_y.inverse_transform(y_test_scaled)
y_pred_orig = scaler_y.inverse_transform(y_pred_scaled)
r2_values = r2_score(y_test_orig, y_pred_orig, multioutput='raw_values')
avg_r2 = np.mean(r2_values[r2_values >= -1])
print("\n=== Model Accuracy Evaluation ===")
print(f"Overall Avg R²: {avg_r2:.4f}")


# --- 8. User Prediction Input ---
print("\n" + "=" * 50)
print("USER-EDITABLE PREDICTION INPUTS")
print("=" * 50)

initial_discovery_inputs = {
    'P_PERIOD': 300,            # Orbital Period (days)
    'P_RADIUS': 1.400,           # Planet Radius (Earth radii)
    'P_TEMP_EQUIL': 270.0,     # Equilibrium Temperature (K)
    'S_TEMPERATURE': 5778.0,    # Star Temperature (K)
    'S_MASS': 1,              # Star Mass (Solar masses)
    'P_COMPOSITION_ROCKY': 1.0  # 1.0 = Rocky, 0.0 = Gaseous
}

# Load trained system and predict
system = load_system()
if system:
    X_new_df, y_pred_df = predict_exoplanet_properties(initial_discovery_inputs, system)
else:
    print("⚠️ Could not load system. Using in-memory objects instead.")
    system = {
        'model': model,
        'scaler_X': scaler_X,
        'scaler_y': scaler_y,
        'X_train_columns': X_train.columns.tolist(),
        'logged_columns': logged_columns,
        'target_columns': target_columns
    }
    X_new_df, y_pred_df = predict_exoplanet_properties(initial_discovery_inputs, system)


# --- 9. Display Final Prediction Report ---
print("\n" + "=" * 50)
print(f"PREDICTION REPORT | Overall Model R²: {avg_r2:.4f}")
print("=" * 50)

input_vals = X_new_df.iloc[0].round(4).to_dict()
output_vals = y_pred_df.iloc[0].round(4).to_dict()
max_key_len = max(len(k) for k in (list(input_vals.keys()) + list(output_vals.keys())))

print("--- INPUTS USED ---")
for k, v in input_vals.items():
    print(f"[Input]     {k:<{max_key_len}}: {v:.4f}")

print("\n--- PREDICTED OUTPUTS ---")
for k, v in output_vals.items():
    print(f"[Predicted] {k:<{max_key_len}}: {v:.4f}")


CSV loaded: Shape (5599, 118)

--- Model Configuration ---
Inputs (Minimal Initial + Composition Hint, 6): ['P_PERIOD', 'P_RADIUS', 'P_TEMP_EQUIL', 'S_TEMPERATURE', 'S_MASS'] + P_COMPOSITION_ROCKY
Targets (Derived/Hard, 39): P_ECCENTRICITY is included. Other examples: ['P_MASS', 'P_SEMI_MAJOR_AXIS', 'P_ECCENTRICITY', 'P_INCLINATION', 'S_DISTANCE']...
Added P_COMPOSITION_ROCKY to training data based on P_DENSITY.

Training LightGBM MultiOutputRegressor (Fast Mode)...
✅ System components saved to ONE file: 'exoplanet_predictor_system.pkl'

=== Model Accuracy Evaluation ===
Overall Avg R²: 0.8897

USER-EDITABLE PREDICTION INPUTS

✅ Loaded model system from 'exoplanet_predictor_system.pkl'
Expected 7 input features:
['P_PERIOD', 'P_RADIUS', 'P_TEMP_EQUIL', 'S_TEMPERATURE', 'S_MASS', 'P_COMPOSITION_ROCKY', 'water_potential_proxy']

PREDICTION REPORT | Overall Model R²: 0.8897
--- INPUTS USED ---
[Input]     P_PERIOD             : 300.0000
[Input]     P_RADIUS             : 1.4000
[Input]   